In [1]:
! pip install -U spacy

In [16]:
import os
import pprint
import string
import re
import pandas as pd
import plotly.express as px

import nltk
from nltk.text import Text
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from nltk.collocations import BigramCollocationFinder,BigramAssocMeasures
import spacy
from spacy import displacy

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split

nltk.download('all')
pp = pprint.PrettyPrinter(indent=4, compact=True)
nlp = spacy.load("en_core_web_lg")

# Stop words del inglés
english_stops = set(stopwords.words('english'))
english_stops.update(string.punctuation)

[nltk_data] Error loading all: <urlopen error [Errno -3] Temporary
[nltk_data]     failure in name resolution>


In [3]:
descr_df = pd.read_csv('../input/ted-talks/ted_main.csv')
trans_df = pd.read_csv('../input/ted-talks/transcripts.csv')

In [4]:
merged_df = pd.merge(descr_df, trans_df, on="url")
merged_df.head()

,comments,description,duration,event,film_date,languages,main_speaker,name,num_speaker,published_date,ratings,related_talks,speaker_occupation,tags,title,url,views,transcript
0,4553,Sir Ken Robinson makes an entertaining and pro...,1164,TED2006,1140825600,60,Ken Robinson,Ken Robinson: Do schools kill creativity?,1,1151367060,"[{'id': 7, 'name': 'Funny', 'count': 19645}, {...","[{'id': 865, 'hero': 'https://pe.tedcdn.com/im...",Author/educator,"['children', 'creativity', 'culture', 'dance',...",Do schools kill creativity?,https://www.ted.com/talks/ken_robinson_says_sc...,47227110,Good morning. How are you?(Laughter)It's been ...
1,265,With the same humor and humanity he exuded in ...,977,TED2006,1140825600,43,Al Gore,Al Gore: Averting the climate crisis,1,1151367060,"[{'id': 7, 'name': 'Funny', 'count': 544}, {'i...","[{'id': 243, 'hero': 'https://pe.tedcdn.com/im...",Climate advocate,"['alternative energy', 'cars', 'climate change...",Averting the climate crisis,https://www.ted.com/talks/al_gore_on_averting_...,3200520,"Thank you so much, Chris. And it's truly a gre..."
2,124,New York Times columnist David Pogue takes aim...,1286,TED2006,1140739200,26,David Pogue,David Pogue: Simplicity sells,1,1151367060,"[{'id': 7, 'name': 'Funny', 'count': 964}, {'i...","[{'id': 1725, 'hero': 'https://pe.tedcdn.com/i...",Technology columnist,"['computers', 'entertainment', 'interface desi...",Simplicity sells,https://www.ted.com/talks/david_pogue_says_sim...,1636292,"(Music: ""The Sound of Silence,"" Simon & Garfun..."
3,200,"In an emotionally charged talk, MacArthur-winn...",1116,TED2006,1140912000,35,Majora Carter,Majora Carter: Greening the ghetto,1,1151367060,"[{'id': 3, 'name': 'Courageous', 'count': 760}...","[{'id': 1041, 'hero': 'https://pe.tedcdn.com/i...",Activist for environmental justice,"['MacArthur grant', 'activism', 'business', 'c...",Greening the ghetto,https://www.ted.com/talks/majora_carter_s_tale...,1697550,If you're here today — and I'm very happy that...
4,593,You've never seen data presented like this. Wi...,1190,TED2006,1140566400,48,Hans Rosling,Hans Rosling: The best stats you've ever seen,1,1151440680,"[{'id': 9, 'name': 'Ingenious', 'count': 3202}...","[{'id': 2056, 'hero': 'https://pe.tedcdn.com/i...",Global health expert; data visionary,"['Africa', 'Asia', 'Google', 'demo', 'economic...",The best stats you've ever seen,https://www.ted.com/talks/hans_rosling_shows_t...,12005869,"About 10 years ago, I took on the task to teac..."


In [5]:
def remove_stop_words(text):
  """
    Remueve stop words en inglés

    Attributes
    ----------
    text: list
      lista de palabras (tokens) a filtrar

    Returns
    -------
    list
      lista de palabras sin los stop words
  """
  return [token for token in text if token.lower() not in english_stops]

In [6]:
def lematize_words(text):
  """
    Lematización de palabras - aplica lematización de palabras sobre un set de tokens

    Attributes
    ----------
    text: list
      lista de palabras (tokens) sobre los cuales se aplicará la lematización
    
    Returns
    -------
    list
      lista con todas las lematizaciones de las palabras
  """
  doc = nlp(" ".join(text))
  return [token.lemma_ for token in doc]

In [7]:
def stem_words(text):
  """
    Stemming de palabras - aplica stemming de palabras sobre un set de tokens

    Attributes
    ----------
    text: list
      lista de palabras (tokens) sobre las que se aplicará stemming
    
    Returns
    -------
    list
      lista con los stems de las palabras
  """
  stemmer = SnowballStemmer('english')
  return [stemmer.stem(word) for word in text]

In [8]:
def remove_meaningless_words(text):
  """
    Remueve palabras sin significado

    Attributes
    ----------
    text: list
      lista de palabras (tokens) a filtrar
    
    Returns
    -------
    list
      lista de palabras filtrada en base a expresiones regulares
  """
  patterns = [r"(^={1,}=$)", r'\u200b']
  tokens = text
  for pattern in patterns:
    regexp = re.compile(pattern)
    tokens = [token for token in tokens if not regexp.search(token)]
  return tokens

In [9]:
def tokenize(text, mode='word'): 
  """
    Tokenización de documento - tokeniza un documento por palabra o por oración

    Attributes
    ----------
    text: str
      documento a tokenizar
    mode: str, optional
      método de tokenización (default: 'word' (por palabra))
    
    Returns
    -------
    list
      lista de tokens 
    
    Raises
    ------
      Exception
        si el mode no es 'word' o 'sentence'
  """
  if mode == 'word':
    return word_tokenize(text, language='english')
  elif mode == 'sentence':
    return sent_tokenize(text, language='english')
  else:
    raise Exception('metodo de tokenizacion no encontrado')

In [14]:
def pre_procesamiento_texto(text):
  """
    Pre-procesamiento y obtención de las 20 palabras más significativas

    Attributes
    ----------
    text: str
      documento a analizar

    Returns
    -------
    pd.DataFrame
      retorna un dataframe con las 20 palabras que más se repiten y su frecuencia
  """
  tokenized = tokenize(text)
  without_stops = remove_stop_words(tokenized)
  meaningfull_tokens = remove_meaningless_words(without_stops)
  lematized_words = lematize_words(meaningfull_tokens)
  # stemmed_words = stem_words(lematized_words
  return lematized_words

In [17]:
# TODO: Replace merged_df.speaker_occupation with the actual tag/tags used for clustering
X_train, X_test, y_train, y_test = train_test_split(merged_df.transcript, merged_df.speaker_occupation, test_size=.30)
tfidf_vectorizer = TfidfVectorizer(tokenizer=pre_procesamiento_texto)
X_train_transform = tfidf_vectorizer.fit_transform(X_train)

In [19]:
X_train_transform

<1726x61815 sparse matrix of type '<class 'numpy.float64'>'
	with 831674 stored elements in Compressed Sparse Row format>